In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def load_KP(path = "datasets/KP24.txt"):
    with open(path, 'r') as f:
        inputData = f.readlines()
    
    Capacity = int(inputData[0].strip())
    N = int(inputData[1].strip())
    item_list=[]
    for item in inputData[2:]:
        #del '\n' 
        item = item.strip()
        #split by ' '
        item = item.split(' ')
        #(p, w)
        item_list.append((int(item[0]), int(item[1])))

    return Capacity, N, item_list

In [18]:
def DistanceMatrix(cities, n):
    dis_matrix = np.zeros([n,n])

    for i in range(n):
        for j in range(i+1, n):
            a = np.array(cities[i])
            b = np.array(cities[j])
            c = a - b
            dis_matrix[i, j] = np.sqrt(np.sum(c*c))
            dis_matrix[j, i] = dis_matrix[i, j]

    return dis_matrix

In [16]:
def load_TSP(path = "datasets/eil51.txt"):
    with open(path, 'r') as f:
        inputData = f.readlines()
    
    N = int(inputData[0].strip())
    node_list=[]
    for node in inputData[1:]:
        #del '\n' 
        node = node.strip()
        #split by ' '
        node = node.split(' ')
        node_list.append((int(node[1]), int(node[2])))

    dis_matrix = DistanceMatrix(node_list, N)
    
    return node_list, N, dis_matrix

In [19]:
node_list, num_cities, dis_matrix = load_TSP()
print(node_list)
print(num_cities)

[(37, 52), (49, 49), (52, 64), (20, 26), (40, 30), (21, 47), (17, 63), (31, 62), (52, 33), (51, 21), (42, 41), (31, 32), (5, 25), (12, 42), (36, 16), (52, 41), (27, 23), (17, 33), (13, 13), (57, 58), (62, 42), (42, 57), (16, 57), (8, 52), (7, 38), (27, 68), (30, 48), (43, 67), (58, 48), (58, 27), (37, 69), (38, 46), (46, 10), (61, 33), (62, 63), (63, 69), (32, 22), (45, 35), (59, 15), (5, 6), (10, 17), (21, 10), (5, 64), (30, 15), (39, 10), (32, 39), (25, 32), (25, 55), (48, 28), (56, 37), (30, 40)]
51


In [7]:
Capacity, N, item_list = load_KP()
print(Capacity)
print(N)
print(item_list)

6404180
24
[(825594, 382745), (1677009, 799601), (1676628, 909247), (1523970, 729069), (943972, 467902), (97426, 44328), (69666, 34610), (1296457, 698150), (1679693, 823460), (1902996, 903959), (1844992, 853665), (1049289, 551830), (1252836, 610856), (1319836, 670702), (953277, 488960), (2067538, 951111), (675367, 323046), (853655, 446298), (1826027, 931161), (65731, 31385), (901489, 496951), (577243, 264724), (466257, 224916), (369261, 169684)]


In [8]:
D = num_cities - 1
if D < N: D = N

In [14]:
def decode_TSP(indiv, n = num_cities - 1):
    decode = []
    for i in range(n):
        decode.append((i+1, indiv[i]))
    decode.sort(key=lambda tup: tup[1], reverse=False)
    sol = [i[0] for i in decode]
    return np.array(sol)


def decode_KP(indiv, n = N):
    sol = []
    for i in indiv[:n]:
        if i < 0.5: sol.append(0)
        else: sol.append(1)
    return np.array(sol)


In [27]:
list(decode_TSP([0.4, 0.7, 0.2, 0.5], n = 4))

[3, 1, 4, 2]

In [15]:
decode_KP([0.4, 0.7, 0.2, 0.5], n = 4)

array([0, 1, 0, 1])

In [ ]:
import math

def cost_tsp(indiv):
    n = len(indiv) - 1
    cost = dis_matrix[0, indiv[0]]
    for i in range(n):
        cost += dis_matrix[indiv[i], indiv[i+1]]
    cost += dis_matrix[indiv[n], 0]

    return cost

def cost_kp(indiv, n = N):
    cost = 0
    weight_total = 0

    for i in range(n):
        cost += indiv[i]*item_list[i][0]
        weight_total += indiv[i]*item_list[i][1]
 
    if(weight_total > Capacity):
        cost -= 50*(weight_total - Capacity)
    return cost

def Factorial_Cost(indiv, t):
    if(t == 0):
        decode = decode_TSP(indiv)
        return(decode)
    else:
        decode = decode_KP(indiv)
        return(decode)